# Example of Data Preprocessing with already completed data
Use already completed data for the table join process.

In [10]:
import polars as pl
import numpy as np
from sklearn.datasets import fetch_openml

from pytred import DataHub
from pytred import DataNode
from pytred.annotations import polars_table

## Make Dataset
This example uses the titanic dataset.  
A column named "record_id" is added as a Key to join data.

In [11]:
X, y = fetch_openml("titanic", version=1, as_frame=True, return_X_y=True)
df_titanic = pl.from_pandas(X).with_columns(
    survived=y.values.astype(int),
    record_id=np.arange(len(X)),
)
display(df_titanic)

pclass,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest,survived,record_id
i64,str,cat,f64,i64,i64,str,f64,str,cat,str,f64,str,i64,i64
1,"""Allen, Miss. E…","""female""",29.0,0,0,"""24160""",211.3375,"""B5""","""S""","""2""",null,"""St Louis, MO""",1,0
1,"""Allison, Maste…","""male""",0.9167,1,2,"""113781""",151.55,"""C22 C26""","""S""","""11""",null,"""Montreal, PQ /…",1,1
1,"""Allison, Miss.…","""female""",2.0,1,2,"""113781""",151.55,"""C22 C26""","""S""",null,null,"""Montreal, PQ /…",0,2
1,"""Allison, Mr. H…","""male""",30.0,1,2,"""113781""",151.55,"""C22 C26""","""S""",null,135.0,"""Montreal, PQ /…",0,3
1,"""Allison, Mrs. …","""female""",25.0,1,2,"""113781""",151.55,"""C22 C26""","""S""",null,null,"""Montreal, PQ /…",0,4
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
3,"""Zabour, Miss. …","""female""",14.5,1,0,"""2665""",14.4542,null,"""C""",null,328.0,null,0,1304
3,"""Zabour, Miss. …","""female""",null,1,0,"""2665""",14.4542,null,"""C""",null,null,null,0,1305
3,"""Zakarian, Mr. …","""male""",26.5,0,0,"""2656""",7.225,null,"""C""",null,304.0,null,0,1306


## Preprocessing list

Implement a DataHub class that performs these preprocessing tasks
1. replace "male" with 1 and "female" with 0
2. fill in missing values of age with the mean
3. sum of sibsp and parch
4. onehot encoding of embarked

In this example, we have a table that has already been preprocessed, and we will demonstrate a case where only joining is performed. In this case, the DataNode class is utilized.

### Preparing the Preprocessed Table

In [15]:
# replace "male" with 1 and "female" with 0
replaced_sex = df_titanic.select(
    "record_id",
    "sex",
    sex_replaced=pl.when(pl.col("sex") == "male").then(1).otherwise(0),
)

# fill in missing values of age with the mean
age_mean = df_titanic.select("age").mean().to_numpy()[0][0]

filled_age = df_titanic.select(
    "record_id",
    "age",
    filled_age=pl.col("age").fill_null(age_mean),
)

# sum of sibsp and parch
cnt_family = df_titanic.select(
    "record_id",
    "sibsp",
    "parch",
    cnt_family=pl.col("sibsp") + pl.col("parch"),
)

# onehot encoding of embarked
onehot_embarked = df_titanic.select(
    "record_id",
    "embarked",
    embarked_C=pl.when(pl.col("embarked") == "C").then(1).otherwise(0),
    embarked_Q=pl.when(pl.col("embarked") == "Q").then(1).otherwise(0),
    embarked_S=pl.when(pl.col("embarked") == "S").then(1).otherwise(0),
)

## Associating table joining information with each table

The DataNode class can manage data, its management name within DataHub, and the methods for joining.


In [19]:
dn_replaced_sex = DataNode(
    table=replaced_sex,
    keys=["record_id"],
    join="left",
    name="replaced_sex",
)

dn_filled_age = DataNode(
    table=filled_age,
    keys=["record_id"],
    join="left",
    name="filled_age",
)

dn_cnt_family = DataNode(
    table=cnt_family,
    keys=["record_id"],
    join="left",
    name="cnt_family",
)

dn_onehot_embarked = DataNode(
    table=onehot_embarked,
    keys=["record_id"],
    join="left",
    name="onehot_embarked",
)

## Preprocessing Data

In [22]:
data_hub = DataHub(
    df_titanic.select("record_id"),
    dn_replaced_sex,
    dn_filled_age,
    dn_cnt_family,
    dn_onehot_embarked,
)

output = data_hub()

display(output)

record_id,sex,sex_replaced,age,filled_age,sibsp,parch,cnt_family,embarked,embarked_C,embarked_Q,embarked_S
i64,cat,i32,f64,f64,i64,i64,i64,cat,i32,i32,i32
0,"""female""",0,29.0,29.0,0,0,0,"""S""",0,0,1
1,"""male""",1,0.9167,0.9167,1,2,3,"""S""",0,0,1
2,"""female""",0,2.0,2.0,1,2,3,"""S""",0,0,1
3,"""male""",1,30.0,30.0,1,2,3,"""S""",0,0,1
4,"""female""",0,25.0,25.0,1,2,3,"""S""",0,0,1
…,…,…,…,…,…,…,…,…,…,…,…
1304,"""female""",0,14.5,14.5,1,0,1,"""C""",1,0,0
1305,"""female""",0,null,29.881135,1,0,1,"""C""",1,0,0
1306,"""male""",1,26.5,26.5,0,0,0,"""C""",1,0,0
